## Inheritance: for good or for worse


### Subclassing built-in types is tricky


In [6]:
# setitem
# 复写父函数行为
class DoppelDict(dict):
    def __setitem__(self, key, val):
        super().__setitem__(key, [val]*2)

In [2]:
dd = DoppelDict(one=1)
dd

{'one': 1}

In [3]:
dd['two'] = 2

In [4]:
dd

{'one': 1, 'two': [2, 2]}

In [5]:
dd.update(three=3)
dd

{'one': 1, 'three': 3, 'two': [2, 2]}

可以看到：

- settitem 作为指定函数，但却没有改变 'one' 的 value
- dd['two'] 直接使用了指定，所以成功的复写了父函数
- update 依然为改变父函数行为


In [7]:
# getitem
# 

class AnswerDict(dict):
    def __getitem__(self, key):
        return 42
    


In [11]:
# ad['a']　语句调用了 AnswerDict.__getitem__ 方法
ad = AnswerDict(a='foo')
ad['a']

42

In [12]:
# 但在 dict 中存在的仍然是　{'a':'foo'} 键值对
d = {}
d.update(ad)
d['a']


'foo'

In [10]:
d

{'a': 'foo'}

In [13]:
# 针对以上的两个问题，可以使用 UserDict 解决

import collections

class Doppledict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2)
        
        

In [14]:
dd2 = Doppledict2(one=1)
dd2

{'one': [1, 1]}

In [15]:
dd2['two'] = 2
dd2

{'two': [2, 2], 'one': [1, 1]}

In [16]:
dd2.update(three=3)
dd2

{'two': [2, 2], 'three': [3, 3], 'one': [1, 1]}

In [17]:
# gititem 也一样

class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42

In [18]:
ad2 = AnswerDict2(one=1)
ad2

{'one': 1}

In [19]:
ad2['one']

42

In [20]:
d = {}
d.update(ad2)
d

{'one': 42}

### Multiple inheritance and method resolution order

多重继承的执行顺序


In [25]:
class Ａ:
    
    def ping(self):
        print('A-ping:', self)
        
class B(A):
    def pong(self):
        print('B-pong:', self)
        
class C(A):
    
    def pong(self):
        print('C-pong:', self)
        
class D(B, C):
    
    def ping(self):
        super().ping()
        print('D-post-ping:', self)
        
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)
        

In [27]:
# 执行 pong　时，按照继承顺序，先从 B 父类中寻找
d = D()
d.pong()

B-pong: <__main__.D object at 0x7f81a4473d30>


In [28]:
d.ping()

A-ping: <__main__.D object at 0x7f81a4473d30>
D-post-ping: <__main__.D object at 0x7f81a4473d30>


In [29]:
d.pingpong()

A-ping: <__main__.D object at 0x7f81a4473d30>
D-post-ping: <__main__.D object at 0x7f81a4473d30>
A-ping: <__main__.D object at 0x7f81a4473d30>
B-pong: <__main__.D object at 0x7f81a4473d30>
B-pong: <__main__.D object at 0x7f81a4473d30>
C-pong: <__main__.D object at 0x7f81a4473d30>


In [31]:
# 执行顺序
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

### Multiple inheritance in the real world

tkinter 架构图

![](Selection_002.jpg)

In [32]:
# 使用 mro 查看继承顺序
import tkinter

def print_mro(cls):
    print(','.join(c.__name__ for c in cls.__mro__))

In [33]:
print_mro(tkinter.Toplevel)

Toplevel,BaseWidget,Misc,Wm,object


In [34]:
print_mro(tkinter.Text)

Text,Widget,BaseWidget,Misc,Pack,Place,Grid,XView,YView,object


### Provide aggregate classes to users


```

class Wigets(E, F, H, K):
    
    pass


# 这种代码的意义在于，用户不需要去逐一记忆 EFHK　中的方法
# 仅仅使用 Wigets 类即可

```